# Discovery of brown dwarfs mining the 2MASS and SDSS database

## Introduction 

This tutorial is based on the EURO-VO tutorial of the same name, which can be found here: http://www.euro-vo.org/sites/default/files/documents/tutorial-brown-dwarfs_2019Apr.pdf 

Brown dwarfs are objects occupying the gap between the least massive stars and the most massive planets. They are intrinsically faint objects so their detection is not straightforward and, in fact, was almost impossible until the advent of global surveys at deep optical and near-infrared bands like SDSS, 2MASS or DENIS among others. We propose here to mine the 2MASS point source catalogue (2MASS-PSC) and SDSS-DR9 databases to identify T-type brown dwarfs through an appropriate combination of colours in the optical and the infrared, an approach that perfectly fits into the Virtual Observatory.

## Goals of this tutorial
In this use case, we explore different ways to do the same tasks with different VO tools. These tasks
include:
 - obtaining data from the SDSS and 2MASS catalogues in a given sky region,
 - crossmatching the results of these searches,
 - filtering the resulting table for brown dwarfs, and
 - verifying our sample of brown dwarfs.

Software packages needed for this tutorial are Aladin, TOPCAT, STILTS and Python (or you may just use the Binder in your web browser).

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.samp import SAMPIntegratedClient
from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import sigma_clipped_stats
import astropy.visualization as ap_vis

from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch
from astroquery.simbad import Simbad
import ipyaladin as ipyal
import pyvo

import warnings 
warnings.filterwarnings('ignore')


## Discovery
We start by searching for the 2MASS point source catalogue (2MASS-PSC) and the SDSS data release 9 catalogue (SDSS DR9). To find and query catalogues, there are two packages we can use: pyVO and astroquery. We will show pyVO for searching the 2MASS-PSC and astroquery for SDSS DR9. 

With the pyVo package we are using the TAP service of VizieR. TAP is a Virtual Observatory protocol to query databases of tables. The language we use to compile the query is called ADQL and is similar to SQL. For more details on ADQL see also the dedicated VO tutorial (http://docs.g-vo.org/adql-gaia/html/). 

In [ ]:
tap_vizier = pyvo.dal.TAPService('http://tapvizier.u-strasbg.fr/TAPVizieR/tap')
mass_psc_set = tap_vizier.search("SELECT  *  FROM tables " + 
                                 "WHERE description LIKE '%2MASS%Cutri%'").to_table()
mass_psc_set['table_name', 'description']

As mentioned above we are interested in the 2MASS-pSC, which has the table_name `II/246/out` in VizieR. Before we move on to construct a more complicated query, we have a quick look at the 2MASS-PSC table and all the available columns. We do so by downloading the first 5 rows:

In [ ]:
mass_psc_head = tap_vizier.search("SELECT TOP 5 * FROM \"II/246/out\" ").to_table()
mass_psc_head

As in the original tutorial, we only want to get data within a 14arcmin radius of RA=08h30m00s, Dec=01d30m00s. ADQL has a function, which allows to run an astronomical cone search query. From the quick inital inspection of the table we now know that we will need the columns `RAJ2000` and `DEJ2000` to only get sources at a certain location. Since we want to get colours for our Brown Dwarf candidates, we will also want to get the `Jmag`, `Hmag` and `Kmag` columns. For a different analysis you might want to choose other columns. 

In [ ]:
coord = SkyCoord(ra='08:30:00', dec='01:30:00', unit=(u.hourangle, u.deg))
mass_psc = tap_vizier.search(
    "SELECT * " +
    "FROM \"II/246/out\" as tm_psc " + 
    "WHERE 1=CONTAINS(" + 
    "POINT(\'ICRS\', tm_psc.RAJ2000, tm_psc.DEJ2000)," +
    "CIRCLE(\'ICRS\', {0:.3f}, {1:.3f}, 14/60))".format(coord.ra.value,
                                                        coord.dec.value)).to_table()
mass_psc

Now that we have all the data we want from the 2MASS-PSC, we move on to query SDSS DR9 with astroquery. First we want to find out which tables are available:

In [ ]:
catalog_list_sdss = Vizier.find_catalogs('SDSS DR9')
for k, v in catalog_list_sdss.items():
    print(k, ': ', v.description)

We want to get data from the catalogue called `V/139`. As before let's have a look at a few rows (per default `Vizier.get_catalogs` gives you 50 rows) of the code to figure out, which ones are most interesting for us.

In [ ]:
test_sdss = Vizier.get_catalogs('V/139')
print(test_sdss)
test_sdss[0]

Again we want to get the coordinates of the source as well as magnitudes at all five SDSS bands (u, g, r, i  and z), and a classification (cl). Do restrict the query to returning these columns and to get more than 50 rows returned, we create a custom `Vizier` object and then use this object to again query within 14arcmin of RA=08h30m00s, Dec=01d30m00s.

In [ ]:
custom_vizier = Vizier(row_limit=-1, 
                       columns=['RA_ICRS', 'DE_ICRS', 'umag', 'gmag', 'rmag', 
                                'imag', 'zmag', 'cl'])
coord = SkyCoord(ra='08:30:00', dec='01:30:00', unit=(u.hourangle, u.deg))
sdss_results = custom_vizier.query_region(coord, radius=14 * u.arcmin, 
                                          catalog='V/139')
sdss_results

So we have one table in the resulting lists of tables. It is called `V/139/sdss9`. Let's have a look at it:

In [ ]:
sdss_results[0]

## Cross-matching
In this next step, we find common sources in the 2MASS-PSC and SDSS-DR9 catalogues. One simple way to do this using the CDS XMatch service, which can also be reached with `astroquery`. The most reliable way to quuery XMatch is by uploading one of the tables (which will also make sure we stay in our region of interest) and crossmatching it to a table available in VizieR. 

In [ ]:
mass_psc.write('Data/2MASS_PSC.vot', format='votable', overwrite=True)
sdss_results[0].write('Data/SDSS_DR9.vot', format='votable', overwrite=True)

In [ ]:
sdss_mass = XMatch.query(cat1=open('Data/2MASS_PSC.vot'),
                         cat2='vizier:V/139/sdss9',
                         max_distance=4 * u.arcsec, 
                         colRA1='RAJ2000', colDec1='DEJ2000')
sdss_mass

For the SDSS DR9 table this action has given us all the columns. So before we move on, we restrict the table to the columns, we are actually interested in. 

In [ ]:
sdss_mass = sdss_mass['2MASS', 'RAJ2000', 'DEJ2000', 'Jmag', 'Hmag', 'Kmag', 
                      'SDSS9', 'RAdeg', 'DEdeg', 'umag', 'gmag', 'rmag', 
                      'imag', 'zmag', 'cl']

print('We are currently working with {} sources'.format(len(sdss_mass)))

## Filtering
In this step we select all those sources from our cross-matched catalogue that have colours and fluxes as expected for brown dwarfs. 

The selection cirteria for Brown Dwarfs are:
 - `umag > 22.0` --> magnitude in u-band fainter than 22.0mag
 - `gmag > 22.2` --> magnitude in g-band fainter than 22.2mag
 - `Jmag - Hmag < 0.3` --> J-H band colour smaller than 0.3mag
 - `Hmag - Kmag < 0.3` --> H-K band colour smaller than 0.3mag

We now apply these selection criteria to our table of point sources. 

In [ ]:
# Add columsn with J-H and H-K colour 
sdss_mass['J-H'] = sdss_mass['Jmag'] - sdss_mass['Hmag']
sdss_mass['H-K'] = sdss_mass['Hmag'] - sdss_mass['Kmag']
# Limit sources to those that fulfil the selection criteria
index_bd = (sdss_mass['umag'] > 22.0) & (sdss_mass['gmag'] > 22.2) & \
    (sdss_mass['J-H'] < 0.3) & (sdss_mass['H-K'] < 0.3) 
candidates = sdss_mass[index_bd]
print('We have {} candidates for Brown Dwarfs'.format(len(candidates)))

In [ ]:
candidates

Note that we are using slightly different selection criteria here in the notebook than in the original tutorial. The reason is the following. The CDS XMatch service uses VizieR tables with their default filtering. For the SDSS DR 9 table, VizieR will filter to only use primary sources and disregard secondary sources, which is not the case when downloading the VizieR table with e.g. TOPCAT. Now the primary SDSS source for this brown dwarf is classified as an extended source by the automated SDSS pipeline, probably because it is hardly detected and very faint. So if we remove all extended sources we also throw away the target of our studies. Hence, the filtering in this notebook is less strict to allow 

## Visualising the Brown Dwarf candidates
The ipyaladin package allows to run an instance of Aladin Lite in your Jupyter notebook. 

In [ ]:
aladin1 = ipyal.Aladin(survey='P/SDSS9/color', fov=0.03, 
                       target='{0:.4f} {1:.4f}'.format(candidates['RAJ2000'][0], 
                                                       candidates['DEJ2000'][0]))
aladin1

As with any instance of Aladin Lite you can zoom in and out by scrolling your mouse. In the top left corner of the widget, you may chose which survey to look at, add layers, search for sources, ... 

You can also add tables to the widget by:

In [ ]:
aladin1.add_table(candidates)

If you now click on the small square indicating the location of the source, the widget will show the corresponding table values in the lower part of the widget. 

As you can see at one of the locations hardly anything is visible and at the location of the other brown dwarf candidate, there is a bright star. To further investigate these two sources, we now query Simbad for them. 

## Verifying the the Brown Dwarf candidate
In the Simbad database, an object type is assigned to every entry. So if this Brown Dwarf candidate is in the Simbad database, we might learn more about the object type:

In [ ]:
customSimbad = Simbad()
customSimbad.remove_votable_fields('coordinates')
customSimbad.add_votable_fields('ra(2;A;ICRS;J2000;2000)', 
                                'dec(2;D;ICRS;J2000;2000)', 
                                'otype')
candidate_coord = SkyCoord(ra=candidates['RAJ2000'][0],
                           dec=candidates['DEJ2000'][0], 
                           unit=u.deg)
candidate_simbad = customSimbad.query_region(candidate_coord, radius=10 * u.arcsec)
candidate_simbad

In [ ]:
customSimbad = Simbad()
customSimbad.remove_votable_fields('coordinates')
customSimbad.add_votable_fields('ra(2;A;ICRS;J2000;2000)', 
                                'dec(2;D;ICRS;J2000;2000)', 
                                'otype')
candidate_coord = SkyCoord(ra=candidates['RAJ2000'][1],
                           dec=candidates['DEJ2000'][1], 
                           unit=u.deg)
candidate_simbad = customSimbad.query_region(candidate_coord, radius=1 * u.arcmin)
candidate_simbad

Indeed the first candidate is classified as a brown dwarf. Within 10arcsec of the second source, no entry is found in Simbad although this is the brighter star. The reason for this might be the following: sources in Simbad are sources that have individually been discussed in the literature. If no papers have been published specifcially about this object, it will not appear in Simbad. However, as the object was included in the SDSS tables it is included in VizieR through the table it belongs to. 

If we now want to see, in which papers the Brown Dwarf 2MASS J08304878+0128311 was included, we may use the following query in Simbad.

In [ ]:
tap_simbad = pyvo.dal.TAPService('http://simbad.u-strasbg.fr:80/simbad/sim-tap')
result = tap_simbad.search("SELECT BIBCode, " +
                      "       Journal, " +
                      "       Title, " +
                      "       \"year\", " +
                      "       Volume, " +
                      "       Page || '-' || Last_Page AS \"Pages\", " +
                      "       DOI " +
                      "FROM ref JOIN has_ref ON oidbibref = oidbib JOIN ident ON has_ref.oidref = ident.oidref " +
                      "WHERE id = '2MASS J08304878+0128311' " +
                      "ORDER BY \"year\" DESC; ").to_table()
result

For further reading you could now go to ADS (https://ui.adsabs.harvard.edu/) and search for the publications via their bibcode (enter in the search field `bibcode:...`) or their doi (`doi:...`).